Requirement

In [3]:
%pip install --upgrade pip
'''
%pip install pandas
%pip install numpy
%pip install pdfplumber
%pip install tqdm
%pip install requests
%pip install beautifulsoup4
%pip install lxml
%pip install langchain-text-splitters
'''
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


PHASE A

PHASE 1A: Scraping News

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm 

def scrape_unhas_smart_tiles():
    url = "https://www.unhas.ac.id/berita/?lang=id"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers,  timeout=10)
    soup = BeautifulSoup(response.text, 'lxml')
    
    articles = soup.find_all('a', class_='jet-smart-tiles__box-link')
    
    news_list = []
    
    print(f"Mencoba mencari dengan class 'jet-smart-tiles__box-link'...")
    print(f"Ditemukan {len(articles)} elemen.")

    for link_tag in tqdm(articles, desc="Scraping Berita"):
        title = link_tag.get('aria-label')
        link = link_tag.get('href')

        if title and link:
            news_list.append({
                'title': title,
                'source': link
            })
            
    return news_list

hasil = scrape_unhas_smart_tiles()

if hasil:
    df = pd.DataFrame(hasil)
    print("\nBerhasil mengambil data!")
    print(df.head())
    df.to_csv('../data/processed/dataset_berita_unhas.csv', index=False)
else:
    print("\nError saat mengambil data.")

Mencoba mencari dengan class 'jet-smart-tiles__box-link'...
Ditemukan 33 elemen.


Scraping Berita: 100%|██████████| 33/33 [00:00<?, ?it/s]


Berhasil mengambil data!
                                               title  \
0  Mau Kuliah di Unhas? Ayo. Hadiri Unhas Open Da...   
1  22 Lulusan Terbaik Unhas Raih Penghargaan, Dom...   
2  Rektor Unhas Tekankan Penguatan Budaya Ilmiah ...   
3  Sebanyak 1.365 Wisudawan Baru Pada Wisuda Unha...   
4  Komisi Nasional Disabilitas Berkunjung ke Unha...   

                                              source  
0  https://www.unhas.ac.id/mau-kuliah-di-unhas-ay...  
1  https://www.unhas.ac.id/22-lulusan-terbaik-unh...  
2  https://www.unhas.ac.id/rektor-unhas-tekankan-...  
3  https://www.unhas.ac.id/sebanyak-1-365-wisudaw...  
4  https://www.unhas.ac.id/komisi-nasional-disabi...  


In [6]:
import time

# df = pd.DataFrame(hasil) 
# Kalau sudah hilang, load dari CSV: df = pd.read_csv('../data/processed/dataset_berita_unhas.csv')

def get_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.text, 'lxml')
        
        content_div = soup.find('div', class_='elementor-widget-theme-post-content')
        
        if content_div:
            paragraphs = content_div.find_all('p')
            full_text = ' '.join([p.text.strip() for p in paragraphs if p.text.strip()])
            return full_text
        else:
            # Cadangan: struktur standar WordPress
            backup_div = soup.find('div', class_='entry-content')
            if backup_div:
                paragraphs = backup_div.find_all('p')
                return ' '.join([p.text.strip() for p in paragraphs])
            
            return "" 
    except Exception as e:
        return f"Error: {e}"

print("Mulai mengambil isi berita satu per satu...")

full_contents = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row['source']
    
    content = get_content(url)
    full_contents.append(content)
    
    # agar tidak dianggap ddos
    time.sleep(1)
    
df['content'] = full_contents
df['type'] = 'berita'

# filter hasil
df_filter = df[df['content'].str.len() > 50]
print("\n--- SELESAI! ---")
print(df_filter[['title', 'content']].head())
df_filter.to_csv('../data/processed/dataset_berita_unhas_lengkap.csv', index=False)
print("Data lengkap disimpan ke 'dataset_unhas_lengkap.csv'")

Mulai mengambil isi berita satu per satu...


100%|██████████| 33/33 [01:42<00:00,  3.12s/it]


--- SELESAI! ---
                                               title  \
0  Mau Kuliah di Unhas? Ayo. Hadiri Unhas Open Da...   
1  22 Lulusan Terbaik Unhas Raih Penghargaan, Dom...   
2  Rektor Unhas Tekankan Penguatan Budaya Ilmiah ...   
3  Sebanyak 1.365 Wisudawan Baru Pada Wisuda Unha...   
4  Komisi Nasional Disabilitas Berkunjung ke Unha...   

                                             content  
0  Universitas Hasanuddin mengundang para siswa S...  
1  Universitas Hasanuddin menggelar prosesi Wisud...  
2  Rektor Unhas, Prof. Dr. Ir. Jamaluddin Jompa, ...  
3  Universitas Hasanuddin kembali menyelenggaraka...  
4  Universitas Hasanuddin melalui Pusat Disabilit...  
Data lengkap disimpan ke 'dataset_unhas_lengkap.csv'


PHASE 2A: Cleaning News

In [8]:
import pandas as pd
import re

# 1. LOAD DATASET MENTAH
df = pd.read_csv("../data/processed/dataset_berita_unhas_lengkap.csv")

# 2. DEFINISI FUNGSI PEMBERSIH (REGEX)
def clean_news_text(text):
    # a. Pastikan input adalah string (jaga-jaga ada nilai NaN/Null)
    text = str(text)
    
    # b. Hapus Newline, Tab, Return (Ganti dengan 1 spasi)
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    
    # c. Hapus pola "Baca Juga: ..."
    text = re.sub(r'Baca [Jj]uga.*?(?=\.|$)', '', text, flags=re.IGNORECASE)
    
    # d. Hapus inisial penulis/fotografer di dalam kurung
    # Contoh: (*/mir), (*/dhs), (Humas Unhas), (img: dok)
    text = re.sub(r'\s*\([^)]*?Unhas.*?\)', '', text, flags=re.IGNORECASE) # Spesifik Unhas
    text = re.sub(r'\s*\(\*/.*?\)', '', text) # Pola inisial wartawan (*/...)
    
    # e. Hapus "Editor :" atau "Penulis :" di akhir berita sampai habis
    text = re.sub(r'(Editor|Penulis)\s*[:|].*?$', '', text, flags=re.IGNORECASE)

    # f. Hapus Spasi Berlebih (Double space jadi single space)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 3. TERAPKAN PEMBERSIHAN
print("Sedang membersihkan teks berita (Regex)...")
tqdm.pandas(desc="Cleaning Content")
df['content'] = df['content'].progress_apply(clean_news_text)

# 4. FILTERING KUALITAS (HAPUS BARIS SAMPAH)
print("Sedang memfilter data yang terlalu pendek...")
jumlah_awal = len(df)
df_clean = df[df['content'].str.len() > 50].copy()
jumlah_akhir = len(df_clean)
dibuang = jumlah_awal - jumlah_akhir

print(f"   -> Data awal: {jumlah_awal} baris")
print(f"   -> Data dibuang: {dibuang} baris (Terlalu pendek/kosong)")
print(f"   -> Data akhir: {jumlah_akhir} baris")

# 5. SIMPAN HASIL AKHIR
output_file = '../data/processed/dataset_berita_unhas_clean.csv'
df_clean.to_csv(output_file, index=False)

print(f"\nSUKSES! Dataset bersih tersimpan di: {output_file}")

# 6. CEK CONTOH DATA (PREVIEW)
print("\n--- Contoh 1 Data Teratas ---")
print("Judul  :", df_clean.iloc[0]['title'])
print("Konten (Akhir): ...", df_clean.iloc[0]['content'][-150:])

Sedang membersihkan teks berita (Regex)...


Cleaning Content: 100%|██████████| 33/33 [00:00<00:00, 2565.18it/s]

Sedang memfilter data yang terlalu pendek...
   -> Data awal: 33 baris
   -> Data dibuang: 0 baris (Terlalu pendek/kosong)
   -> Data akhir: 33 baris

SUKSES! Dataset bersih tersimpan di: ../data/processed/dataset_berita_unhas_clean.csv

--- Contoh 1 Data Teratas ---
Judul  : Mau Kuliah di Unhas? Ayo. Hadiri Unhas Open Day 2026
Konten (Akhir): ... i daerah. Bagi calon peserta yang ingin mengikuti kegiatan ini, proses pendaftaran dapat dilakukan melalui tautan https://s.unhas.ac.id/RegistUOD2026.


PHASE B

PHASE 1B: Extract PDF to TEXT

In [9]:
import os
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re

# 1. Lokasi Tesseract.exe
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 2. Lokasi Poppler (Folder bin)
POPPLER_PATH = r'C:\poppler-25.12.0\Library\bin' 

# 3. Folder Data
FOLDER_PDF = '../data/pdf/'
OUTPUT_CSV = '../data/processed/dataset_peraturan_unhas.csv'

def clean_ocr_text(text):
    text = re.sub(r'\n', ' ', text)       # Hapus enter
    text = re.sub(r'\s+', ' ', text)      # Hapus spasi ganda
    text = re.sub(r'[^a-zA-Z0-9.,/:;()%\- ]', '', text) # Hapus karakter aneh
    return text.strip()

def extract_text_with_ocr(pdf_path):
    print(f"-> Mengonversi PDF ke Gambar & OCR: {os.path.basename(pdf_path)}...")
    full_text = ""
    
    try:
        # 1. Ubah PDF jadi List Gambar (Per Halaman)
        images = convert_from_path(pdf_path, poppler_path=POPPLER_PATH)
        
        # 2. Baca tiap halaman pakai Tesseract
        for i, image in enumerate(images):
            page_text = pytesseract.image_to_string(image, lang='eng') 
            full_text += page_text + " "
            
    except Exception as e:
        print(f"Error OCR: {e}")
        return ""
        
    return full_text

data_pdf = []

if not os.path.exists(FOLDER_PDF):
    print(f"Folder '{FOLDER_PDF}' tidak ditemukan.")
else:
    files = [f for f in os.listdir(FOLDER_PDF) if f.lower().endswith('.pdf')]
    print(f"Mulai OCR untuk {len(files)} dokumen...")

    for filename in tqdm(files, desc="Proses OCR PDF"):
        filepath = os.path.join(FOLDER_PDF, filename)
        
        # Proses OCR 
        raw_text = extract_text_with_ocr(filepath)
        
        # Cleaning
        clean_text = clean_ocr_text(raw_text)
        
        if len(clean_text) > 50:
            data_pdf.append({
                'title': filename,
                'source': f"Dokumen PDF ({filename})",
                'content': clean_text,
                'type': 'pdf'
            })
            print(f"      ✅ Selesai: {len(clean_text)} karakter.")

    # Simpan
    if data_pdf:
        df = pd.DataFrame(data_pdf)
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"\nSUKSES! Data tersimpan di 'dataset_peraturan_unhas.csv'")

Mulai OCR untuk 5 dokumen...


Proses OCR PDF:   0%|          | 0/5 [00:00<?, ?it/s]

-> Mengonversi PDF ke Gambar & OCR: ACADEMIC_REGULATION_cfecbc8cb1.pdf...


Proses OCR PDF:  20%|██        | 1/5 [01:02<04:08, 62.07s/it]

      ✅ Selesai: 70492 karakter.
-> Mengonversi PDF ke Gambar & OCR: ATURAN REKTOR - Pedoman Tata Cara Penyimpanan Karya Ilmiah pada Repositori Universitas Hasanuddin 2024-2.pdf...


...

      ✅ Selesai: 9668 karakter.
-> Mengonversi PDF ke Gambar & OCR: SK Rektor tentang Peraturan Pemanfaatan Koleksi Perpustakaan Unhas (2).pdf...


Proses OCR PDF:  80%|████████  | 4/5 [01:37<00:18, 18.27s/it]

      ✅ Selesai: 7258 karakter.
-> Mengonversi PDF ke Gambar & OCR: SOP KEANGGOTAAN.pdf...


Proses OCR PDF: 100%|██████████| 5/5 [01:42<00:00, 20.50s/it]

      ✅ Selesai: 2764 karakter.

SUKSES! Data tersimpan di 'dataset_peraturan_unhas.csv'


PHASE 2B: Cleaning PDF

In [10]:
import pandas as pd
import re

# 1. Load Data 
input_file = '../data/processed/dataset_peraturan_unhas.csv'
df = pd.read_csv(input_file)

# 2. Definisi Kamus Typo 
KAMUS_TYPO = {
    "SKEPUTUSAN": "KEPUTUSAN",
    "SKEP": "KEP",
    "mt ": "",      # Hapus 'mt' (biasanya noise logo)
    "ae ": "a. ",   # Perbaiki bullet point a
    "bs ": "b. ",   # Perbaiki bullet point b
    "cs ": "c. ",   # Perbaiki bullet point c
    "ds ": "d. ",
    "Menimbang :": "\nMenimbang :", # Kasih enter biar rapi
    "Mengingat :": "\nMengingat :",
    "Memutuskan :": "\nMemutuskan :",
    "MEMUTUSKAN": "\nMEMUTUSKAN",
    "BABI": "BAB I",
    "BAB Ill": "BAB III",
    "BAB Il": "BAB II",
}

def advanced_cleaning(text):
    if not isinstance(text, str): return ""
    
    # 1. Hapus Metadata Web
    text = re.sub(r'Baca [Jj]uga.*?(?=\.|$)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s*\([^)]*?Unhas.*?\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(Editor|Penulis)\s*[:|].*?$', '', text, flags=re.IGNORECASE)
    
    # 2. Perbaikan Typo OCR 
    for salah, benar in KAMUS_TYPO.items():
        text = text.replace(salah, benar)
        
    # 3. Fix Bullet Points
    text = re.sub(r'(?m)^\s*([a-z])\s+', r'\1. ', text)

    # 4. Final Trim (Hanya hapus spasi/tab berlebih)
    text = re.sub(r'[ \t]+', ' ', text) 
    text = re.sub(r'\n\s*\n', '\n', text) 
    
    return text.strip()

# 3. Eksekusi
print("Mulai membersihkan typo dan artifact...")
content = df['content'].copy()
df['content'] = df['content'].apply(advanced_cleaning)

# 4. Cek Hasil
idx = content.str.contains("SKEPUTUSAN", na=False)
if idx.any():
    target_idx = idx[idx].index[0]
    print("\n--- CONTOH PERBAIKAN ---")
    print("SEBELUM:", content[target_idx][:100]) 
    print("SESUDAH:", df.loc[target_idx, 'content'][:100])
else:
    print("\nTidak menemukan 'SKEPUTUSAN'.")

# 5. Simpan Hasil Akhir
df.to_csv('../data/processed/dataset_peraturan_unhas_clean.csv', index=False)
print("\n✅ Data bersih tersimpan di 'dataset_peraturan_unhas_clean.csv'")

Mulai membersihkan typo dan artifact...

--- CONTOH PERBAIKAN ---
SEBELUM: - SKEPUTUSAN mt REKTOR UNIVERSITAS HASANUDDIN NOMOR : 609/114/P/2007  TENTANG : PERATURAN PEMANFAATA
SESUDAH: - KEPUTUSAN REKTOR UNIVERSITAS HASANUDDIN NOMOR : 609/114/P/2007 TENTANG : PERATURAN PEMANFAATAN KOL

✅ Data bersih tersimpan di 'dataset_peraturan_unhas_clean.csv'


PHASE 3: MERGING

In [11]:
import pandas as pd
import os

print("Sedang menggabungkan dataset Berita dan PDF...")

# 1. Load Data Berita 
try:
    df_berita = pd.read_csv('../data/processed/dataset_berita_unhas_clean.csv')
    print(f"✅ Data Berita: {len(df_berita)} baris")
except FileNotFoundError:
    print("⚠️ Data Berita tidak ditemukan! Jalankan Phase 1 & 2 dulu.")
    df_berita = pd.DataFrame()

# 2. Load Data PDF 
try:
    df_pdf = pd.read_csv('../data/processed/dataset_peraturan_unhas.csv')
    print(f"✅ Data PDF: {len(df_pdf)} dokumen")
except FileNotFoundError:
    print("ℹ️ Data PDF belum ada (Mungkin folder kosong). Skip.")
    df_pdf = pd.DataFrame()

# 3. Gabungkan 
if not df_pdf.empty:    
    # Pilih kolom yang sama saja
    cols = ['title', 'source', 'content', 'type']
    df_combined = pd.concat([df_berita[cols], df_pdf[cols]], ignore_index=True)
else:
    df_combined = df_berita

# 4. Simpan Master Dataset
output_master = '../data/processed/dataset_master_unhas.csv'
df_combined.to_csv(output_master, index=False)

print(f"\n🎉 TOTAL DATASET: {len(df_combined)} baris.")
print(f"Tersimpan di 'dataset_master_unhas.csv'")

Sedang menggabungkan dataset Berita dan PDF...
✅ Data Berita: 33 baris
✅ Data PDF: 5 dokumen

🎉 TOTAL DATASET: 38 baris.
Tersimpan di 'dataset_master_unhas.csv'


PHASE 4: CHUNKING & EXPORT TO JSONL

In [12]:
import pandas as pd
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

# 1. Load Data Gabungan (Merged CSV)
input_file = '../data/processed/dataset_master_unhas.csv' 
print(f"Membaca data dari 'dataset_master_unhas.csv'...")
df = pd.read_csv(input_file)

# 2. Konfigurasi Chunker
# - chunk_size=1000: Setiap potongan sekitar 1000 karakter (cukup untuk 1-2 paragraf)
# - chunk_overlap=200: Ada irisan 200 karakter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""] # Prioritas pemotongan
)

# 3. Proses Chunking
print("Sedang melakukan chunking...")
jsonl_data = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Ambil teks
    original_text = str(row['content']) # Pastikan string
    source= row['source']
    title = row['title']
    
    # Lakukan pemotongan
    chunks = text_splitter.split_text(original_text)
    
    # Simpan setiap potongan
    for i, chunk_text in enumerate(chunks):
        entry = {
            "id": f"{row['type']}_{index}_{i}",          
            "title": title,
            "source": source,
            "text": chunk_text              
        }
        jsonl_data.append(entry)

# 4. Simpan ke JSONL 
output_file = '../output/knowledge_base_unhas.jsonl'

with open(output_file, 'w', encoding='utf-8') as f:
    for entry in jsonl_data:
        json.dump(entry, f)
        f.write('\n')

print(f"\n🎉 SELESAI! Data siap RAG.")
print(f"   - Total Dokumen Asli : {len(df)}")
print(f"   - Total Chunks (Potongan) : {len(jsonl_data)}")
print(f"   - Tersimpan di : {output_file}")

# 5. Preview Hasil
print("\n--- CONTOH CHUNK (JSONL) ---")
print(json.dumps(jsonl_data[0], indent=2))

Membaca data dari 'dataset_master_unhas.csv'...
Sedang melakukan chunking...


100%|██████████| 38/38 [00:00<00:00, 9308.70it/s]


🎉 SELESAI! Data siap RAG.
   - Total Dokumen Asli : 38
   - Total Chunks (Potongan) : 241
   - Tersimpan di : ../output/knowledge_base_unhas.jsonl

--- CONTOH CHUNK (JSONL) ---
{
  "id": "berita_0_0",
  "title": "Mau Kuliah di Unhas? Ayo. Hadiri Unhas Open Day 2026",
  "source": "https://www.unhas.ac.id/mau-kuliah-di-unhas-ayo-hadiri-unhas-open-day-2026/?lang=id",
  "text": "Universitas Hasanuddin mengundang para siswa Sekolah Menengah Atas (SMA), khususnya yang saat ini duduk di kelas XII, untuk menghadiri Unhas Open Day (UOD) 2026. Kegiatan ini akan berlangsung selama 2 hari, yaitu Sabtu dan Minggu, 7 dan 8 Februari 2026, di GOR JK Arenatorium, mulai pukul 08.00 WITA. UOD 2026 merupakan ajang memperkenalkan kampus secara lebih dekat kepada calon mahasiswa dan masyarakat luas. Wakil Rektor Bidang Akademik dan Kemahasiswaan, Prof. drg. Muhammad Ruslin, M.Kes., Ph.D., Sp.BM(K), menjelaskan UOD 2026 merupakan bagian penting dari komitmen Unhas dalam menghadirkan layanan informasi pendid